<a href="https://colab.research.google.com/github/anweshaban21/Portfolio_Agents/blob/main/Volatility.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yfinance as yf
import numpy as np

def get_volatility(ticker, period="1y"):
    data = yf.download(ticker, period=period, progress=False)
    if data.empty:
        return 0
    data['returns'] = np.log(data['Close'] / data['Close'].shift(1))
    volatility = np.std(data['returns'].dropna()) * np.sqrt(252)
    return volatility



In [ ]:
import pandas as pd
def get_beta(stock_ticker, market_ticker="^GSPC", period="1y"):
    stock_data = yf.download(stock_ticker, period=period, progress=False)
    market_data = yf.download(market_ticker, period=period, progress=False)

    if stock_data.empty or market_data.empty:
        return 0  # No data, no beta

    stock_returns = np.log(stock_data['Close'] / stock_data['Close'].shift(1)).dropna()
    market_returns = np.log(market_data['Close'] / market_data['Close'].shift(1)).dropna()

    if stock_returns.empty or market_returns.empty:
        return 0  # No returns, no beta

    returns_data = pd.DataFrame({
        'stock': stock_returns,
        'market': market_returns
    }).dropna()

    if returns_data.empty:
        return 0  # Again, no valid data to calculate beta

    covariance = returns_data.cov().iloc[0, 1]
    market_variance = returns_data['market'].var()

    if market_variance == 0:
        return 0  # Avoid division by zero

    beta = covariance / market_variance
    return beta



In [ ]:
def get_risk_score(ticker, market_ticker="^GSPC", period="1y"):
    volatility = get_volatility(ticker, period)
    beta = get_beta(ticker, market_ticker, period)

    # Normalization (assume reasonable ranges)
    volatility_score = min(volatility / 0.5, 1)  # Assume max 50% volatility
    beta_score = min(abs(beta) / 2, 1)            # Assume max beta = 2

    # Weighted average
    risk_score = (0.6 * volatility_score) + (0.4 * beta_score)
    return risk_score


In [ ]:
tickers = ["AAPL", "MSFT", "TSLA", "AMZN"]

for ticker in tickers:
    risk = get_risk_score(ticker)
    print(f"{ticker} Risk Score: {risk:.2f}")


ValueError: If using all scalar values, you must pass an index